In [ ]:
# Install Whisper if not installed
!pip install openai-whisper

import whisper
import warnings
import os
import pandas as pd
from transformers import LEDTokenizer, LEDForConditionalGeneration
import gdown
from google.colab import drive


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=99e53ab32c42a07691ffb94256982f94d9fa1939a8608b6a100091dda5d97455
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [ ]:
import warnings
import gdown
import pandas as pd
from transformers import LEDTokenizer, LEDForConditionalGeneration
import whisper
import re

# Mount Google Drive (optional, needed if saving files to Google Drive)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    print("Google Drive not available. Proceeding without mounting.")

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="FP16 is not supported on CPU; using FP32 instead")
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using torch.load with weights_only=False")
warnings.filterwarnings("ignore", category=UserWarning, message="huggingface_hub cache-system uses symlinks")

# Initialize Whisper and LED models
model_whisper = whisper.load_model("medium")
tokenizer_led = LEDTokenizer.from_pretrained("allenai/led-base-16384")
model_led = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384")

# Data structure to hold titles, transcripts, and summaries
data = {'title': [], 'Text': [], 'Summary': []}

# Function to chunk text by sentences for summarization
def chunk_text_by_sentences(text, max_length=1024):
    sentences = text.split('. ')
    chunk, current_length = [], 0
    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length <= max_length:
            chunk.append(sentence)
            current_length += sentence_length
        else:
            yield '. '.join(chunk) + '.'
            chunk, current_length = [sentence], sentence_length
    if chunk:
        yield '. '.join(chunk) + '.'

# Function to remove repetitive phrases
def remove_repetitions(text):
    pattern = r'(\b\w+\b.*?\b\w+\b)(?:\s+\1)+'
    return re.sub(pattern, r'\1', text, flags=re.IGNORECASE)

# Google Drive file ID (from the link provided)
file_id = 'YOUR_FILE_ID'

# Generate the download URL
download_url = f'https://drive.google.com/uc?id={file_id}'

# Download the file using gdown
output_path = '/content/whisper_audio.mp3'  # Path where the audio file will be saved
gdown.download(download_url, output_path, quiet=False)

# Process the downloaded audio file
try:
    # Transcribe audio using Whisper
    result = model_whisper.transcribe(output_path, language="en", temperature=0.7)
    transcript = result["text"]

    # Clean the transcript to remove repetitive phrases
    transcript = remove_repetitions(transcript)
    print("Cleaned Transcript:", transcript)

    # Summarize transcript in chunks
    chunked_summaries = []
    for chunk in chunk_text_by_sentences(transcript, max_length=1024):
        inputs = tokenizer_led(chunk, return_tensors="pt", truncation=True, max_length=16384)
        summary_ids = model_led.generate(
            inputs["input_ids"],
            max_length=150,
            min_length=30,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        chunked_summary = tokenizer_led.decode(summary_ids[0], skip_special_tokens=True)
        chunked_summaries.append(chunked_summary)

    # Combine chunked summaries into one summary
    summary = " ".join(chunked_summaries)

except Exception as e:
    print(f"Error occurred: {e}")
    summary = "Error during summarization."
    transcript = "Error during transcription."

# Add to data dictionary
data['title'].append("whisper_audio.mp3")
data['Text'].append(transcript)
data['Summary'].append(summary)

# Save data to CSV
df = pd.DataFrame(data)
df.to_csv("/content/audio_summary.csv", index=False)
print("Transcriptions and summaries saved to /content/audio_summary.csv")

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/648M [00:00<?, ?B/s]

Downloading...
From: https://drive.google.com/uc?id=10n-BDbkr8VdNx8pTsq3vvH_-vVDYhhxk
To: /content/whisper_audio.mp3

100%|██████████| 13.9M/13.9M [00:00<00:00, 188MB/s]
Input ids are automatically padded from 1214 to 2048 to be a multiple of `config.attention_window`: 1024


Cleaned Transcript:  Artificial Intelligence and Machine Learning. What's the difference? Are they the same? Well, some people kind of frame the question this way. It's A.I. vs. ML. Is that the right way to think of this? Or is it A.I. equals ML? Or is it A.I. is somehow something different than ML? So here's three equations. I wonder which one is going to be right. Well, Let's talk about this. First of all, when we talk about AI, I think it's important to come with definitions because a lot of people have different ideas of what this is. So I'm going to assert the simple definition that AI is basically exceeding or matching the capabilities of a human. So we're trying to match the intelligence, whatever that means, and capabilities of a human subject. Now, what could that involve? There's a number of different things. For instance, one of them is the ability to discover, to find out new information. Another is the ability to infer, to read in information from other sources that maybe 